In [1]:
# import libraries
import pandas as pd
import datetime
#from geopy import 

# set view parameters
pd.set_option('display.max_columns', 999)

# import crash data (dtype declaration fixes import error)
df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv', dtype={3:object})

# clean up column names
df.rename(columns={'CRASH DATE': 'crash_date',
                   'CRASH TIME': 'crash_time',
                   'BOROUGH': 'borough',
                   'ZIP CODE': 'zip_code',
                   'LATITUDE': 'latitude',
                   'LONGITUDE': 'longitude',
                   'LOCATION': 'location',
                   'ON STREET NAME': 'street_name',
                   'CROSS STREET NAME': 'cross_street',
                   'OFF STREET NAME': 'address',
                   'NUMBER OF PERSONS INJURED': 'persons_injured',
                   'NUMBER OF PERSONS KILLED': 'persons_killed',
                   'NUMBER OF PEDESTRIANS INJURED': 'pedestrians_injured',
                   'NUMBER OF PEDESTRIANS KILLED': 'pedestrians_killed',
                   'NUMBER OF CYCLIST INJURED': 'cyclists_injured',
                   'NUMBER OF CYCLIST KILLED': 'cyclists_killed',
                   'NUMBER OF MOTORIST INJURED': 'motorists_injured',
                   'NUMBER OF MOTORIST KILLED': 'motorists_killed',
                   'CONTRIBUTING FACTOR VEHICLE 1': 'contrib_factor_vehicle_1',
                   'CONTRIBUTING FACTOR VEHICLE 2': 'contrib_factor_vehicle_2',
                   'CONTRIBUTING FACTOR VEHICLE 3': 'contrib_factor_vehicle_3',
                   'CONTRIBUTING FACTOR VEHICLE 4': 'contrib_factor_vehicle_4',
                   'CONTRIBUTING FACTOR VEHICLE 5': 'contrib_factor_vehicle_5',
                   'COLLISION_ID': 'collision_id',
                   'VEHICLE TYPE CODE 1': 'vehicle_type_1',
                   'VEHICLE TYPE CODE 2': 'vehicle_type_2',
                   'VEHICLE TYPE CODE 3': 'vehicle_type_3',
                   'VEHICLE TYPE CODE 4': 'vehicle_type_4',
                   'VEHICLE TYPE CODE 5': 'vehicle_type_5'}, inplace=True)

# remove NaNs
df.persons_injured.fillna("0", inplace = True)
df.persons_killed.fillna("0", inplace = True)
#df.fillna("", inplace = True)

# convert dtypes
df.crash_date = pd.to_datetime(df.crash_date)
df.crash_time = df.crash_time.astype('datetime64').dt.time
df.persons_injured = df.persons_injured.astype('int64')
df.persons_killed = df.persons_killed.astype('int64')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750704 entries, 0 to 1750703
Data columns (total 32 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   crash_date                datetime64[ns]
 1   crash_time                object        
 2   borough                   object        
 3   zip_code                  object        
 4   latitude                  float64       
 5   longitude                 float64       
 6   location                  object        
 7   street_name               object        
 8   cross_street              object        
 9   address                   object        
 10  persons_injured           int64         
 11  persons_killed            int64         
 12  pedestrians_injured       int64         
 13  pedestrians_killed        int64         
 14  cyclists_injured          int64         
 15  cyclists_killed           int64         
 16  motorists_injured         int64         
 17  motorist

In [2]:
df.isnull().sum()

crash_date                        0
crash_time                        0
borough                      537299
zip_code                     537510
latitude                     207904
longitude                    207904
location                     207904
street_name                  351938
cross_street                 613287
address                     1491134
persons_injured                   0
persons_killed                    0
pedestrians_injured               0
pedestrians_killed                0
cyclists_injured                  0
cyclists_killed                   0
motorists_injured                 0
motorists_killed                  0
contrib_factor_vehicle_1       4907
contrib_factor_vehicle_2     246619
contrib_factor_vehicle_3    1633784
contrib_factor_vehicle_4    1725617
contrib_factor_vehicle_5    1744162
collision_id                      0
vehicle_type_1                 9152
vehicle_type_2               287366
vehicle_type_3              1636915
vehicle_type_4              

In [3]:
# top 10 deadliest ZIP codes for pedestrians
# df.groupby(["borough", "zip_code"]).pedestrians_killed.sum().reset_index().sort_values("pedestrians_killed", ascending=False).head(10)

In [4]:
df['non_drivers_killed'] = df['pedestrians_killed'] + df['cyclists_killed']
df['non_drivers_injured'] = df['pedestrians_injured'] + df['cyclists_injured']
df['non_drivers_total'] = df['non_drivers_injured'] + df['non_drivers_killed']

In [5]:
nonDriversKilled = df.groupby(["borough", "zip_code"]).non_drivers_killed.sum().reset_index().sort_values("non_drivers_killed", ascending=False)
nonDriversKilled.head(10)

,borough,zip_code,non_drivers_killed
170,QUEENS,11354,18
38,BROOKLYN,11214,16
31,BROOKLYN,11207,15
52,BROOKLYN,11229,15
92,MANHATTAN,10025,15
56,BROOKLYN,11233,14
71,MANHATTAN,10002,14
32,BROOKLYN,11208,14
59,BROOKLYN,11236,14
187,QUEENS,11372,14


In [6]:
nonDriversInjured = df.groupby(["borough", "zip_code"]).non_drivers_injured.sum().reset_index().sort_values("non_drivers_injured", ascending=False).head(10)
nonDriversInjured 

,borough,zip_code,non_drivers_injured
50,BROOKLYN,11226,1804
31,BROOKLYN,11207,1603
30,BROOKLYN,11206,1590
71,MANHATTAN,10002,1575
36,BROOKLYN,11212,1501
83,MANHATTAN,10016,1497
35,BROOKLYN,11211,1453
184,QUEENS,11368,1424
27,BROOKLYN,11203,1325
44,BROOKLYN,11220,1262


In [7]:
nonDriversTotal = df.groupby(["borough", "zip_code"]).non_drivers_total.sum().reset_index().sort_values("non_drivers_total", ascending=False).head(10)
nonDriversTotal

,borough,zip_code,non_drivers_total
50,BROOKLYN,11226,1816
31,BROOKLYN,11207,1618
30,BROOKLYN,11206,1602
71,MANHATTAN,10002,1589
36,BROOKLYN,11212,1513
83,MANHATTAN,10016,1505
35,BROOKLYN,11211,1462
184,QUEENS,11368,1432
27,BROOKLYN,11203,1330
44,BROOKLYN,11220,1273
